In [1]:
pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


### Python-dotenv

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

### Chat Models: GPT 3.5 Turbo and GPT-4

In [6]:
from langchain_openai import ChatOpenAI

In [11]:
llm = ChatOpenAI()
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-4-turbo-preview')
print(output.content)

Quantum mechanics is a fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles, characterized by the principles of superposition, quantization, and uncertainty.


In [12]:
help(ChatOpenAI)

Help on class ChatOpenAI in module langchain_openai.chat_models.base:

class ChatOpenAI(langchain_core.language_models.chat_models.BaseChatModel)
 |  ChatOpenAI(*, name: Optional[str] = None, cache: Optional[bool] = None, verbose: bool = None, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, client: Any = None, async_client: Any = None, model: str = 'gpt-3.5-turbo', temperature: float = 0.7, model_kwargs: Dict[str, Any] = None, api_key: Optional[str] = None, base_url: Optional[str] = None, organization: Optional[str] = None, openai_proxy: Optional[str] = None, timeout: Union[float, Tuple[float, float], Any, NoneType] = None, max_retries: int = 2, streaming: bool = False, n: int = 1, max_tokens: Optional[int] = None, tiktoken_model_na

### Chat Completion API

In [13]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

In [14]:
messages = [
    SystemMessage(content='You are a physicist and respond only in Hindi'),
    HumanMessage(content='Explain quantum mechanics in one sentence.'),
]

output = llm.invoke(messages)
print(output.content)

क्वांटम मैकेनिक्स एक भौतिकी शाखा है जो अत्यंत छोटे अणुओं और उनके व्यवहार का अध्ययन करती है, जिसमें स्थिरता, अस्थिरता और अस्पष्टता के सिद्धांत शामिल हैं।


### Caching LLM Response

#### In-Memory Cache

In [15]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [16]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that toddler can understand'
llm.invoke(prompt)

CPU times: user 14.4 ms, sys: 3.97 ms, total: 18.3 ms
Wall time: 1.17 s


"\n\nWhy did the banana go to the doctor? Because it wasn't peeling well!"

In [17]:
%%time
llm.invoke(prompt)

CPU times: user 559 µs, sys: 144 µs, total: 703 µs
Wall time: 714 µs


"\n\nWhy did the banana go to the doctor? Because it wasn't peeling well!"

#### SQLite Cache

In [18]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path='.lanchain.db'))

In [19]:
%%time
llm.invoke('Tell me joke')

CPU times: user 27 ms, sys: 5.43 ms, total: 32.4 ms
Wall time: 920 ms


"\nWhy don't scientists trust atoms?\n\nBecause they make up everything."

In [26]:
%%time
llm.invoke('Tell me joke')

CPU times: user 2.04 ms, sys: 963 µs, total: 3 ms
Wall time: 2.09 ms


"\nWhy don't scientists trust atoms?\n\nBecause they make up everything."

### LLM Streaming

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
# print(llm.invoke(prompt).content) #direct output

In [28]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
In the dead of night, when the sky is black
The moon shines bright, casting shadows back
A raven calls out, a haunting sound
As it flies high, above the ground

Chorus:
Moon and raven, dance in the night
A mystical sight, a beautiful sight
Their spirits intertwined, forever bound
In the darkness, they are found

Verse 2:
The moon's glow, a silver light
Guiding the raven, through the night
The raven's cry, a mournful tune
Echoing under the pale moon

Chorus:
Moon and raven, dance in the night
A mystical sight, a beautiful sight
Their spirits intertwined, forever bound
In the darkness, they are found

Bridge:
They soar together, in the night sky
A bond unbroken, as the years go by
The moon and raven, a timeless pair
In the darkness, they share

Chorus:
Moon and raven, dance in the night
A mystical sight, a beautiful sight
Their spirits intertwined, forever bound
In the darkness, they are found

Outro:
Moon and raven, a tale so old
Their story forever told
In the night, they will

### Prompt Template

In [31]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are an experienced virologist. 
Write a few sentences about the following virus "{virus}" in {language}'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='odia')
prompt

'You are an experienced virologist. \nWrite a few sentences about the following virus "hiv" in odia'

In [32]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV ହୁମାନ୍ ଇମ୍ଯୁନୋଡେଫିସିୟେନ୍ସି ଭିରସ୍ ଯାହା ମାନବ ଶରୀରର ଇମ୍ୟୁନ୍ ସିଷ୍ଟମ୍ କୁ ଆକ୍ରମଣ କରେ। ଏହାର କାରଣେ ମାନବଙ୍କୁ ଏଇ ଭିରସ୍ ଦ୍ୱାରା ଏଡ୍ସ ରୋଗ ହେଉଥାଏ। ଏହାର ପ୍ରତିରୋଧ ନାହିଁ ଏବଂ ଏହାର ପ୍ରତିକ୍ରିୟା ନାହିଁ।


### ChatPromptTemplates

In [33]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import SystemMessage

In [37]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You response only in json format'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

messages = chat_template.format_messages(n='10', area='Asia')
print(messages)

[SystemMessage(content='You response only in json format'), HumanMessage(content='Top 10 countries in Asia by population.')]


In [36]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "1": {
        "country": "China",
        "population": 1439323776
    },
    "2": {
        "country": "India",
        "population": 1380004385
    },
    "3": {
        "country": "United States",
        "population": 331002651
    },
    "4": {
        "country": "Indonesia",
        "population": 273523615
    },
    "5": {
        "country": "Pakistan",
        "population": 220892340
    },
    "6": {
        "country": "Brazil",
        "population": 212559417
    },
    "7": {
        "country": "Nigeria",
        "population": 206139589
    },
    "8": {
        "country": "Bangladesh",
        "population": 164689383
    },
    "9": {
        "country": "Russia",
        "population": 145934462
    },
    "10": {
        "country": "Mexico",
        "population": 128932753
    }
}


### Simple Chains

In [38]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [39]:
llm = ChatOpenAI()
template = '''You are an experienced virologist. 
Write a few sentences about the following virus "{virus}" in {language}'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='odia')
prompt

'You are an experienced virologist. \nWrite a few sentences about the following virus "hiv" in odia'

In [41]:
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus':'HSV', 'language':'Gujrati'})
print(output)



> Entering new LLMChain chain...
Prompt after formatting:
You are an experienced virologist. 
Write a few sentences about the following virus "HSV" in Gujrati

> Finished chain.
{'virus': 'HSV', 'language': 'Gujrati', 'text': 'HSV એક વાયરસ છે જે હુમન બોડીમાં સંક્રમિત થવાથી હેર્પીસ સીમ્પલેક્સ અને હેર્પીસ ઝોસ્ટર જેવી સમસ્યાઓ ઉંમદી કરે છે. આ વાયરસ વધુમાં વધુ લોકોમાં પ્રકાર 1 અને પ્રકાર 2 દ્વારા પ્રકારિત કરી શકે છે. આ સમસ્યાઓ સામાન્ય રીતે ત્વચા પર ચાલે છે પરંતુ અગર તેની સુરક્ષા ન કરે તો તે અન્ય સમસ્યાઓ પણ ઉંમદી શકે છે.'}


### Sequential Chain

In [42]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

In [43]:
llm1 = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and python programmer. Write a function that implements the concept of {concept}'
)
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

llm2 = ChatOpenAI(model='gpt-4-turbo-preview', temperature=1.2)
prompt_template2 = PromptTemplate.from_template(
    template='Given the python function {function}, describe it as detailed as possible'
)
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)


overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.invoke('linear regression')



> Entering new SimpleSequentialChain chain...
Sure! Here is a simple implementation of linear regression in Python:

```python
import numpy as np

def linear_regression(X, y):
    n = X.shape[0]
    ones = np.ones((n, 1))
    X = np.hstack((ones, X))
    
    beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    
    return beta

# Example usage
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])

beta = linear_regression(X, y)
print("Intercept:", beta[0])
print("Slope:", beta[1])
```

This function takes in a matrix `X` containing the input features and a vector `y` containing the target values. It then calculates the coefficients of the linear regression model using the normal equation method and returns them. The example usage demonstrates how to use the function with sample data.
The provided Python function named `linear_regression` implements the linear regression model calculation utilizing the normal equation method. Linear regression is a fundamental statis

In [44]:
print(output['output'])

The provided Python function named `linear_regression` implements the linear regression model calculation utilizing the normal equation method. Linear regression is a fundamental statistical and machine learning approach used to model the relationship between a dependent variable and one or more independent variables. The goal is to find the linear equation that best fits the distribution of input-output pairs. The overarching aim is predicting the value of the dependent variable for given values of the independent variables.

Here's a step-by-step explanation of how the function works:

### Dependencies
- The function depends on the `numpy` library, which is a popular Python library for numerical computing. It provides efficient data structures for multi-dimensional arrays (`ndarray`) and functions for performing operations on these structures.

### Function Signature
- `linear_regression(X, y)`: The function takes in two parameters:
    - `X`: A 2D numpy array where each row represen

### Langchain Agents in Action: Python REPL

In [45]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [46]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1,100) if n%13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [49]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.invoke('Calculate the sqaure root of the factorial of 12 and display it with 4 decimal points')



> Entering new AgentExecutor chain...
To solve this, I will first calculate the factorial of 12 using the `math` module's `factorial` function. Then, I will calculate the square root of that result using the `sqrt` function from the same module. Finally, I will format the result to display it with 4 decimal points using the `format` function.

Action: Python_REPL
Action Input: import math
Observation: 
Thought:Now that the `math` module is imported, I can calculate the factorial of 12.

Action: Python_REPL
Action Input: factorial_12 = math.factorial(12)
print(factorial_12)
Observation: 479001600

Thought:Now that I have the factorial of 12, I can calculate its square root.

Action: Python_REPL
Action Input: sqrt_factorial_12 = math.sqrt(factorial_12)
print(sqrt_factorial_12)
Observation: 21886.105181141756

Thought:Now that I have the square root of the factorial of 12, I can format it to display with 4 decimal points.

Action: Python_REPL
Action Input: print(f"{sqrt_factorial_12:.4f

{'input': 'Calculate the sqaure root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [50]:
response = agent_executor.invoke('What is the answer to 5.1**7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1**7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [51]:
response

{'input': 'What is the answer to 5.1**7.3?', 'output': '146306.05007233328'}

### LangChain Tools: DuckDuckGo and Wikipedia

In [52]:
#Google, duckduckgo, tavily

In [53]:
pip install -q duckduckgo-search

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires pyqt5<5.16, which is not installed.
spyder 5.4.3 requires pyqtwebengine<5.16, which is not installed.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [58]:
from langchain.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

search = DuckDuckGoSearchRun()
output = search.invoke('Where Subash Chandra Bose born?')
print(output)

Subhas Chandra Bose (/ ʃ ʊ b ˈ h ɑː s ˈ tʃ ʌ n d r ə ˈ b oʊ s / ⓘ shuub-HAHSS CHUN-drə BOHSS; 23 January 1897 - 18 August 1945) was an Indian nationalist whose defiance of British authority in India made him a hero among many Indians, but his wartime alliances with Nazi Germany and Imperial Japan left a legacy vexed by authoritarianism, anti-Semitism, and military failure. Subhas Chandra Bose (born c. January 23, 1897, Cuttack, Orissa [now Odisha], India—died August 18, 1945, Taipei, Taiwan?) was an Indian revolutionary prominent in the independence movement against British rule of India.He also led an Indian national force from abroad against the Western powers during World War II.He was a contemporary of Mohandas K. Gandhi, at times an ally and at other ... Netaji Subhas Chandra Bose was born on 23 January,1897 in Cuttack, Orissa and he died on 18 August,1945, in a hospital in Taiwan after suffering from burn injuries sustained in a plane crash. Subhash Chandra Bose Birth. Subhash Ch

In [56]:
search.name

'duckduckgo_search'

In [57]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [61]:
search = DuckDuckGoSearchResults()
output = search.run('Gopabandhu Das')
print(output)

[snippet: Learn about the life and achievements of Utkalmani Gopabandhu Das, a social reformer, journalist, and freedom fighter from Odisha. He co-founded Utkala Sammilani, established Satyabadi School, and founded The Samaja newspaper., title: Utkalmani Gopabandhu Das - The Gem of Odisha, link: https://incredibleodisha.in/utkalmani-gopabandhu-das-the-gem-of-odisha/], [snippet: Learn about Gopabandhu Das, a visionary leader, social reformer, and nationalist who revolutionized education in Odisha. Discover his achievements, innovations, and challenges in establishing schools, colleges, and universities in the state., title: Gopabandhu Das : The National Education Planner of Odisha, link: https://www.odishalifestyle.com/gopabandhu-das-the-national-education/], [snippet: Gopabandhu Das came into this world on the 9th of October in the year 1877, in the serene village of Suando, situated in close proximity to Puri, within the picturesque state of Odisha. He was born into a Brahmin family, 

In [65]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
wrapper = DuckDuckGoSearchAPIWrapper(region='India', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Jagannath Dham')
print(output)

[snippet: Learn about the history, significance, and rituals of the Jagannath Temple, one of the four Char Dham pilgrimage sites in India. Discover the legends, myths, and culture of Lord Jagannath and his siblings, who are worshipped as manifestations of Lord Vishnu., title: Shree Jagannath Puri Temple - A Complete Visitor's Guide, link: https://www.godearlife.com/jagannath-puri/], [snippet: The Jagannath temple in Puri, Odisha is particularly significant in Vaishnavism, and is regarded as one of the Char Dham pilgrimage sites in India. The Jagannath temple is massive, over 61 metres (200 ft) high in the Nagara architecture style of Hindu temple architecture , and one of the best surviving specimens of Kalinga architecture , namely ..., title: Jagannath - Wikipedia, link: https://en.wikipedia.org/wiki/Jagannath], [snippet: Know the timing for Puri Jagannath Temple Darshan and save time with our guide. Bus, Train, Cab rates and tips for easy darshan. Know the best times and important d

In [68]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f'snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('_'*50)
    print()

snippet: Learn about the history, significance, and rituals of the Jagannath Temple, one of the four Char Dham pilgrimage sites in India. Discover the legends, myths, and culture of Lord Jagannath and his siblings, who are worshipped as manifestations of Lord Vishnu.
Title: Shree Jagannath Puri Temple - A Complete Visitor's Guide
Link: https://www.godearlife.com/jagannath-puri/

__________________________________________________

snippet: The Jagannath temple in Puri, Odisha is particularly significant in Vaishnavism, and is regarded as one of the Char Dham pilgrimage sites in India. The Jagannath temple is massive, over 61 metres (200 ft) high in the Nagara architecture style of Hindu temple architecture , and one of the best surviving specimens of Kalinga architecture , namely ...
Title: Jagannath - Wikipedia
Link: https://en.wikipedia.org/wiki/Jagannath

__________________________________________________

snippet: Know the timing for Puri Jagannath Temple Darshan and save time with 

In [69]:
pip install -q wikipedia

Note: you may need to restart the kernel to use updated packages.


In [70]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [72]:
api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=5000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query':'llamaindex'})

'Page: NebulaGraph\nSummary: NebulaGraph is an open-source distributed graph database built for super large-scale graphs with milliseconds of latency. NebulaGraph adopts the Apache 2.0 license and  also  comes with a wide range of data visualization tools.\n\n\n\nPage: Vector database\nSummary: A vector database, vector store or vector search engine is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from a few hundred to tens of thousands, depending on the complexity of the data being represented. A vector\'s position in this space represents its characterist

In [73]:
wiki.invoke({'query':'Google gemini'})

/Users/rakesh.panigrahy/anaconda3/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/rakesh.panigrahy/anaconda3/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


'Page: Gemini (chatbot)\nSummary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name and developed as a direct response to the meteoric rise of OpenAI\'s ChatGPT, it was launched in a limited capacity in March 2023 before expanding to other countries in May. It was previously based on PaLM, and initially the LaMDA family of large language models.\nLaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI\'s launch of ChatGPT in November 2022 and its subsequent popularity caught Google executives off-guard and sent them into a panic, prompting a sweeping response in the ensuing months. After mobilizing its workforce, the company launched Bard in February 2023, which took center stage during the 2023 Google I/O keynote in May and was upgraded to the Gemini LLM in December. Bard and Duet AI were unified under the Gemi

### Creating a ReAct Agent

In [74]:
pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [75]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [79]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)

template = '''
Answer the following questions as best you can.
Questions: {q}
'''

prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
print(type(prompt))
print(prompt.input_variables)
print(prompt.template)

<class 'langchain_core.prompts.prompt.PromptTemplate'>
['agent_scratchpad', 'input', 'tool_names', 'tools']
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [80]:
# 1. Python REPL tool (for executing python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you wish to use Python to answer the question. You should input Python code'
)


# 2. Wikipedia tool (for searching wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful when you wish to search a topic in wikipedia'
)

# 3.Duckduckgo Search tool (for general web search)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful when you wish to search a topic in web'
)

In [81]:
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]
agent = create_react_agent(llm, tools, prompt)

In [82]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [83]:
question = 'Generate the first 20 numbers in the Fibonacci series'
output = agent_executor.invoke({
    'input':prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
I can write a Python script to generate the first 20 numbers in the Fibonacci series.

Action: Python REPL
Action Input: 
```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        next_fib = fib_series[-1] + fib_series[-2]
        fib_series.append(next_fib)
    return fib_series[:n]

print(fibonacci(20))
```[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
I now know the first 20 numbers in the Fibonacci series.

Final Answer: The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.

> Finished chain.


In [84]:
print(output['input'])


Answer the following questions as best you can.
Questions: Generate the first 20 numbers in the Fibonacci series



In [85]:
print(output['output'])

The first 20 numbers in the Fibonacci series are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.


In [86]:
question = 'Who is the current chief minister of Odisha'
output = agent_executor.invoke({
    'input':prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
I need to find the current chief minister of Odisha, which is a state in India. This information can be quickly found through a web search.

Action: DuckDuckGo Search
Action Input: current chief minister of Odisha
Odisha Full Cabinet Ministers List 2024: A total of 15 ministers along with Odisha CM Mohan Charan Majhi were inducted into the Cabinet on Wednesday (June 12). BJP MLA from Keonjhar Mohan Charan Majhi, who belongs to the tribal Santhal community took oath as Odisha's Chief Minister. The party picked two Deputy CMs — K V Singh Deo and Pravati Parida, who became the first woman Deputy CM of ... In Short. Four-time MLA and tribal leader Mohan Charan Majhi on Wednesday took oath as BJP's first chief minister of Odisha. Governor Raghubar Das administered the oath to Majhi in an event that was attended by Prime Minister Narendra Modi and former Chief Minister Naveen Patnaik. Apart from PM Modi, Union Ministers JP Nadda, Amit Shah ... Mohan Ch

In [87]:
question = 'Tell me about Swami Vivekanand early life'
output = agent_executor.invoke({
    'input':prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To provide information about Swami Vivekananda's early life, I should look up a reliable source that details his biography, such as Wikipedia.

Action: Wikipedia
Action Input: Swami Vivekananda
Page: Swami Vivekananda
Summary: Swami Vivekananda (; Bengali: [ʃami bibekanɔndo] ; IAST: Svāmī Vivekānanda ; 12 January 1863 – 4 July 1902), born Narendranath Datta (Bengali: [nɔrendronatʰ dɔto]), was an Indian Hindu monk, philosopher, author, religious teacher, and the chief disciple of the Indian mystic Ramakrishna. He was a key figure in the introduction of Vedanta and Yoga to the Western world, and is the father of modern Indian nationalism who is credited with raising interfaith awareness and bringing Hinduism to the status of a major world religion in the late nineteenth century.
Born into an aristocratic Bengali Kayastha family in Calcutta, Vivekananda was inclined from a young age towards religion and spirituality. He later found his guru Ramakris